# Day 8: BERT测试和环境验证

测试HuggingFace Transformers库，熟悉BERT的加载和使用


In [ ]:
# Cell 1: 导入库
"""
HuggingFace Transformers库是使用预训练模型的标准工具
提供了BERT、GPT等各种预训练模型
"""

from transformers import BertTokenizer, BertForSequenceClassification, BertModel
import transformers
import torch
import numpy as np

print("Transformers库导入成功")
print(f"Transformers版本: {transformers.__version__}")


Transformers库导入成功
Transformers版本: 4.57.1


In [ ]:
# Cell 2: 加载BERT Tokenizer
"""
Tokenizer的作用：将文本转换为BERT能理解的格式
- 中文BERT使用字符级tokenization（每个汉字是一个token）
- 会添加特殊token：[CLS]、[SEP]、[PAD]等
"""

model_name = 'bert-base-chinese'  # 中文BERT预训练模型

print(f"正在加载 {model_name} 的tokenizer...")
print("（第一次运行会从HuggingFace下载，需要几分钟）")

# 加载tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

print("Tokenizer加载成功")
print(f"词表大小: {tokenizer.vocab_size:,}")  # BERT的词表大小（固定的）


正在加载 bert-base-chinese 的tokenizer...
（第一次运行会从HuggingFace下载，需要几分钟）


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

D:\Anaconda\envs\dl\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ysn\.cache\huggingface\hub\models--bert-base-chinese. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

Tokenizer加载成功
词表大小: 21,128


In [ ]:
# Cell 3: 测试Tokenizer - 基础用法
"""
理解tokenizer如何处理文本：
1. 文本 → Token序列（分词）
2. Token → ID（转换为数字）
3. 添加特殊token：[CLS]、[SEP]
"""

# 测试文本
text = "这是一条体育新闻，中国男篮取得胜利"

print(f"原始文本: {text}")
print("="*50)

# 方法1：tokenize（只分词，返回token）
tokens = tokenizer.tokenize(text)
print(f"\n【Step 1: 分词】")
print(f"Tokens: {tokens}")
print(f"Token数量: {len(tokens)}")

# 方法2：convert_tokens_to_ids（token转ID）
ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"\n【Step 2: Token转ID】")
print(f"IDs: {ids}")

# 方法3：encode（一步到位：文本→ID，会自动添加[CLS]和[SEP]）
encoded = tokenizer.encode(text)
print(f"\n【Step 3: 直接encode】")
print(f"Encoded IDs: {encoded}")
print(f"长度: {len(encoded)}")

# 解码：ID→文本
decoded = tokenizer.decode(encoded)
print(f"\n【Step 4: 解码】")
print(f"Decoded: {decoded}")

print(" 注意：")
print("  - [CLS]: 句子开始标记（ID=101）")
print("  - [SEP]: 句子结束标记（ID=102）")
print("  - BERT的输入必须有这两个特殊token")


原始文本: 这是一条体育新闻，中国男篮取得胜利

【Step 1: 分词】
Tokens: ['这', '是', '一', '条', '体', '育', '新', '闻', '，', '中', '国', '男', '篮', '取', '得', '胜', '利']
Token数量: 17

【Step 2: Token转ID】
IDs: [6821, 3221, 671, 3340, 860, 5509, 3173, 7319, 8024, 704, 1744, 4511, 5074, 1357, 2533, 5526, 1164]

【Step 3: 直接encode】
Encoded IDs: [101, 6821, 3221, 671, 3340, 860, 5509, 3173, 7319, 8024, 704, 1744, 4511, 5074, 1357, 2533, 5526, 1164, 102]
长度: 19

【Step 4: 解码】
Decoded: [CLS] 这 是 一 条 体 育 新 闻 ， 中 国 男 篮 取 得 胜 利 [SEP]
 注意：
  - [CLS]: 句子开始标记（ID=101）
  - [SEP]: 句子结束标记（ID=102）
  - BERT的输入必须有这两个特殊token


In [ ]:
# Cell 4: Tokenizer高级用法 - padding和truncation
"""
实际训练时需要的功能：
- padding: 将短文本补齐到固定长度
- truncation: 将长文本截断
- attention_mask: 告诉BERT哪些是真实内容，哪些是padding
"""

# 准备多条文本（长度不一）
texts = [
    "这是一条短新闻",  # 短
    "这是一条稍微长一点的新闻，包含更多的信息和内容",  # 中
    "这是一条很长很长的新闻，包含了非常多的详细信息，描述了事件的来龙去脉，以及相关的背景知识和专家分析"  # 长
]

print("原始文本长度:")
for i, text in enumerate(texts):
    print(f"  文本{i+1}: {len(text)}字")

# 使用tokenizer的高级功能
encoding = tokenizer(
    texts,
    padding=True,        # 自动padding到batch中最长的长度
    truncation=True,     # 超过max_length就截断
    max_length=20,       # 最大长度设为20（演示用）
    return_tensors='pt'  # 返回PyTorch tensor
)

print(f"\n【编码结果】")
print(f"input_ids shape: {encoding['input_ids'].shape}")
print(f"  解释: [batch_size=3, seq_len=20]")

print(f"\n【input_ids】（Token IDs）:")
print(encoding['input_ids'])

print(f"\n【attention_mask】（注意力掩码）:")
print(encoding['attention_mask'])
print("  解释: 1=真实token, 0=padding")

print(f"\n关键理解:")
print("  - padding: 短文本用[PAD](ID=0)补齐")
print("  - attention_mask: 告诉BERT忽略padding部分")
print("  - truncation: 长文本截断到max_length")


原始文本长度:
  文本1: 7字
  文本2: 23字
  文本3: 49字

【编码结果】
input_ids shape: torch.Size([3, 20])
  解释: [batch_size=3, seq_len=20]

【input_ids】（Token IDs）:
tensor([[ 101, 6821, 3221,  671, 3340, 4764, 3173, 7319,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 6821, 3221,  671, 3340, 4924, 2544, 7270,  671, 4157, 4638, 3173,
         7319, 8024, 1259, 1419, 3291, 1914, 4638,  102],
        [ 101, 6821, 3221,  671, 3340, 2523, 7270, 2523, 7270, 4638, 3173, 7319,
         8024, 1259, 1419,  749, 7478, 2382, 1914,  102]])

【attention_mask】（注意力掩码）:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
  解释: 1=真实token, 0=padding

关键理解:
  - padding: 短文本用[PAD](ID=0)补齐
  - attention_mask: 告诉BERT忽略padding部分
  - truncation: 长文本截断到max_length


In [ ]:
# Cell 5: 加载BERT分类模型
"""
BertForSequenceClassification：
- BERT模型 + 分类头（全连接层）
- 预训练好的BERT + 随机初始化的分类层
- 微调时：BERT参数会更新，分类层从零学习
"""

print(f"正在加载 {model_name} 模型...")
print("（第一次会下载，约400MB，需要几分钟）")

# 加载模型（指定分类类别数）
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=14  # THUCNews有14个类别
)

print("BERT模型加载成功")

# 模型信息
total_params = sum(p.numel() for p in model.parameters())
print(f"\n模型参数量: {total_params/1e6:.1f}M")
print(f"  - BERT层: ~110M参数（预训练好的）")
print(f"  - 分类层: ~11K参数（需要训练）")

print(f"\n模型结构概览:")
print(f"  输入: [batch, seq_len] 的token IDs")
print(f"  BERT编码: → [batch, seq_len, 768]")
print(f"  [CLS]表示: → [batch, 768]")
print(f"  分类层: → [batch, 14]")
print(f"  输出: 14个类别的分数")


正在加载 bert-base-chinese 模型...
（第一次会下载，约400MB，需要几分钟）


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C421DF7850>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 6a14f698-bd4c-4081-b4fe-cea1b3cc8f22)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C42755E620>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: c537e7cc-f9a8-4892-97d7-13dc6b0e7410)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/config.json
Retrying in 2s [Retry 2/5].
Xet Storage is enabl

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT模型加载成功

模型参数量: 102.3M
  - BERT层: ~110M参数（预训练好的）
  - 分类层: ~11K参数（需要训练）

模型结构概览:
  输入: [batch, seq_len] 的token IDs
  BERT编码: → [batch, seq_len, 768]
  [CLS]表示: → [batch, 768]
  分类层: → [batch, 14]
  输出: 14个类别的分数


In [ ]:
# Cell 6: 测试模型推理（未训练的模型）
"""
测试未经微调的BERT能否运行
注意：此时分类层是随机初始化的，预测结果是随机的
微调后才会有准确的预测
"""

# 准备输入
test_text = "股市今日大涨，上证指数突破3000点"

print(f"测试文本: {test_text}")

# Tokenize
encoding = tokenizer(
    test_text,
    padding='max_length',
    truncation=True,
    max_length=128,  # 实际训练时可能用512
    return_tensors='pt'  # 返回PyTorch tensor
)

print(f"\n编码后的格式:")
print(f"  input_ids shape: {encoding['input_ids'].shape}")
print(f"  attention_mask shape: {encoding['attention_mask'].shape}")

# 模型推理
model.eval()  # 设置为评估模式
with torch.no_grad():  # 不计算梯度（推理时）
    outputs = model(**encoding)  # **表示解包字典作为参数
    # 等价于：model(input_ids=encoding['input_ids'], attention_mask=encoding['attention_mask'])

# 查看输出
print(f"\n【模型输出】")
print(f"outputs类型: {type(outputs)}")
print(f"logits shape: {outputs.logits.shape}")
print(f"  解释: [batch_size=1, num_labels=14]")

# 获取预测类别
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()
probs = torch.softmax(logits, dim=1)[0]  # 转为概率

print(f"\n【预测结果】（未训练，随机的）")
print(f"预测类别: {predicted_class}")
print(f"最高概率: {probs[predicted_class]:.4f}")
print(f"\n各类别概率（前5个）:")
for i in range(5):
    print(f"  类别{i}: {probs[i]:.4f}")

print(f"\n 注意：")
print(f"  这是未训练的模型，预测是随机的")
print(f"  微调后，模型会学会正确分类")


测试文本: 股市今日大涨，上证指数突破3000点

编码后的格式:
  input_ids shape: torch.Size([1, 128])
  attention_mask shape: torch.Size([1, 128])

【模型输出】
outputs类型: <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
logits shape: torch.Size([1, 14])
  解释: [batch_size=1, num_labels=14]

【预测结果】（未训练，随机的）
预测类别: 11
最高概率: 0.1599

各类别概率（前5个）:
  类别0: 0.0206
  类别1: 0.0762
  类别2: 0.0517
  类别3: 0.1190
  类别4: 0.0658

 注意：
  这是未训练的模型，预测是随机的
  微调后，模型会学会正确分类


In [ ]:
# Cell 7: BERT vs TextCNN/LSTM - 数据处理对比
"""
理解BERT和传统方法的区别
关键：BERT不需要自己构建词表！
"""

print("="*60)
print("BERT vs 传统方法（TextCNN/LSTM）对比")
print("="*60)

print("\n【传统方法（TextCNN/LSTM）】")
print("  1. 分词: jieba分词")
print("  2. 构建词表: 统计词频，分配ID")
print("  3. 文本→ID: 查词表，未知词用<UNK>")
print("  4. Embedding: 从零学习词向量（或用Word2Vec）")
print("  缺点: 词表需要自己构建，未知词处理不好")

print("\n【BERT方法】")
print("  1. 分词: BertTokenizer（字符级）")
print("  2. 词表: 使用预训练的固定词表（21128个）")
print("  3. 文本→ID: 直接用tokenizer")
print("  4. Embedding: 使用预训练的词向量（已经学好的）")
print("  优点: 不需要构建词表，预训练embedding效果好")

print("\n【具体示例】")
text = "人工智能和深度学习"

# 传统方法（jieba）
import jieba
jieba_tokens = list(jieba.cut(text))
print(f"\njieba分词: {jieba_tokens}")
print(f"  需要: 自己构建词表，'人工智能'可能是未知词")

# BERT方法
bert_tokens = tokenizer.tokenize(text)
print(f"\nBERT分词: {bert_tokens}")
print(f"  说明: 字符级分词，每个字都认识")
print(f"  优势: 没有未知词问题")

print("\n 核心区别:")
print("  - 传统方法: 需要自己建词表（工作量大）")
print("  - BERT: 直接用预训练的（省事！）")
print("  - BERT的embedding已经在大量文本上学习过，效果更好")


BERT vs 传统方法（TextCNN/LSTM）对比

【传统方法（TextCNN/LSTM）】
  1. 分词: jieba分词
  2. 构建词表: 统计词频，分配ID
  3. 文本→ID: 查词表，未知词用<UNK>
  4. Embedding: 从零学习词向量（或用Word2Vec）
  缺点: 词表需要自己构建，未知词处理不好

【BERT方法】
  1. 分词: BertTokenizer（字符级）
  2. 词表: 使用预训练的固定词表（21128个）
  3. 文本→ID: 直接用tokenizer
  4. Embedding: 使用预训练的词向量（已经学好的）
  优点: 不需要构建词表，预训练embedding效果好

【具体示例】


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ysn\AppData\Local\Temp\jieba.cache
Loading model cost 0.434 seconds.
Prefix dict has been built successfully.



jieba分词: ['人工智能', '和', '深度', '学习']
  需要: 自己构建词表，'人工智能'可能是未知词

BERT分词: ['人', '工', '智', '能', '和', '深', '度', '学', '习']
  说明: 字符级分词，每个字都认识
  优势: 没有未知词问题

 核心区别:
  - 传统方法: 需要自己建词表（工作量大）
  - BERT: 直接用预训练的（省事！）
  - BERT的embedding已经在大量文本上学习过，效果更好


In [ ]:
# Cell 8: Batch处理测试
"""
测试BERT处理一个batch的数据
模拟训练时的情况
"""

# 准备一个batch的文本（3条新闻）
batch_texts = [
    "中国男篮在世界杯上取得佳绩",  # 体育
    "股市今日收盘，上证指数上涨",  # 财经
    "最新智能手机发布，搭载AI芯片"   # 科技
]

batch_labels = torch.tensor([0, 13, 11])  # 假设的真实标签

print("Batch数据:")
for i, text in enumerate(batch_texts):
    print(f"  {i+1}. {text}")

# Tokenize整个batch
batch_encoding = tokenizer(
    batch_texts,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

print(f"\nBatch编码:")
print(f"  input_ids shape: {batch_encoding['input_ids'].shape}")
print(f"  解释: [batch_size=3, seq_len=128]")

# 模型推理
model.eval()
with torch.no_grad():
    outputs = model(**batch_encoding)
    logits = outputs.logits  # [3, 14]

# 获取预测
predicted_classes = torch.argmax(logits, dim=1)
print(f"\n【预测结果】（未训练，随机）")
print(f"预测类别: {predicted_classes}")
print(f"真实类别: {batch_labels}")

# 如果训练过，可以计算loss
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(logits, batch_labels)
print(f"\nLoss: {loss.item():.4f}")
print(f"  解释: 这是未训练模型的loss，很大是正常的")
print(f"  微调后loss会降低，准确率会提升")

print("\n Batch处理测试通过")


Batch数据:
  1. 中国男篮在世界杯上取得佳绩
  2. 股市今日收盘，上证指数上涨
  3. 最新智能手机发布，搭载AI芯片

Batch编码:
  input_ids shape: torch.Size([3, 128])
  解释: [batch_size=3, seq_len=128]

【预测结果】（未训练，随机）
预测类别: tensor([ 3, 11, 10])
真实类别: tensor([ 0, 13, 11])

Loss: 2.8547
  解释: 这是未训练模型的loss，很大是正常的
  微调后loss会降低，准确率会提升

 Batch处理测试通过


In [ ]:
# Cell 9: 总结 - BERT使用流程
"""
总结BERT的完整使用流程
为明天的微调训练做准备
"""

print("="*60)
print(" BERT测试完成！核心知识总结")
print("="*60)

print("\n【1. BERT核心组件】")
print("   BertTokenizer: 文本→Token IDs")
print("   BertForSequenceClassification: BERT+分类层")
print("   预训练模型: 已经在大量文本上学习过")

print("\n【2. BERT使用流程】")
print("  Step 1: 加载tokenizer和模型")
print("  Step 2: tokenizer处理文本")
print("     - 分词、转ID、padding、attention_mask")
print("  Step 3: 模型推理/训练")
print("     - 输入: input_ids + attention_mask")
print("     - 输出: logits（14个类别分数）")
print("  Step 4: 获取预测")
print("     - torch.argmax(logits, dim=1)")

print("\n【3. BERT vs 传统方法】")
print("  传统方法:")
print("    - 需要: 分词、建词表、训练embedding")
print("    - 优点: 灵活、可控")
print("    - 缺点: 工作量大、未知词问题")
print("  ")
print("  BERT方法:")
print("    - 需要: 直接用tokenizer")
print("    - 优点: 简单、预训练embedding效果好")
print("    - 缺点: 模型大、推理慢")



print("\n【5. 关键参数记录】")
print(f"  - 模型: bert-base-chinese")
print(f"  - 词表大小: {tokenizer.vocab_size}")
print(f"  - 参数量: ~110M")
print(f"  - Max length: 512（实际训练时）")
print(f"  - 类别数: 14")

print("\n 环境验证完成！准备开始BERT微调！")


🎉 BERT测试完成！核心知识总结

【1. BERT核心组件】
   BertTokenizer: 文本→Token IDs
   BertForSequenceClassification: BERT+分类层
   预训练模型: 已经在大量文本上学习过

【2. BERT使用流程】
  Step 1: 加载tokenizer和模型
  Step 2: tokenizer处理文本
     - 分词、转ID、padding、attention_mask
  Step 3: 模型推理/训练
     - 输入: input_ids + attention_mask
     - 输出: logits（14个类别分数）
  Step 4: 获取预测
     - torch.argmax(logits, dim=1)

【3. BERT vs 传统方法】
  传统方法:
    - 需要: 分词、建词表、训练embedding
    - 优点: 灵活、可控
    - 缺点: 工作量大、未知词问题
  
  BERT方法:
    - 需要: 直接用tokenizer
    - 优点: 简单、预训练embedding效果好
    - 缺点: 模型大、推理慢

【5. 关键参数记录】
  - 模型: bert-base-chinese
  - 词表大小: 21128
  - 参数量: ~110M
  - Max length: 512（实际训练时）
  - 类别数: 14

 环境验证完成！准备开始BERT微调！
